In [ ]:
%load_ext autoreload
%autoreload 2

import bnl
import mir_eval
import plotly
import plotly.express as px
import pandas as pd

In [ ]:
def px_metrics_bar(metrics, title="mir_eval hierarchy metrics"):
    # metrics: dict or OrderedDict of name -> float
    df = pd.Series(metrics, dtype=float).rename_axis("metric").reset_index(name="score")
    fig = px.bar(df, x="metric", y="score", title=title, text="score", range_y=[0,1.2])
    fig.update_traces(texttemplate="%{text:.3f}", textposition="outside")
    fig.update_layout(xaxis_tickangle=-45, width=450, height=300, margin=dict(l=10, r=10, t=50, b=10))
    return fig

In [ ]:
# Get a couple fake reference and estimates going
ref_coarse = bnl.S.from_bs([0, 5, 8, 16, 20])
ref_fine = bnl.S.from_bs([0, 2, 5, 8, 13.3, 16, 18, 20])

In [ ]:
# Bad est 1: right boundaries but spread across multiple layers, and non monotonic.

est = bnl.MS([
    bnl.S.from_bs([0, x.time, 20]) for x in ref_coarse.bs[1:-1]],
    name="est"
).prune_layers()
ref = bnl.MS([ref_coarse, ref_fine], name="ref")
ref.plot().show()
est.plot().show()

In [ ]:
est_mono = est.contour("depth").level().to_ms()
est_mono.plot()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

for window in [5, 10, 15, None]:
    result = mir_eval.hierarchy.evaluate(ref.itvls, ref.labels, est.itvls, est.labels, window=window)
    px_metrics_bar(result, title=f"window = {window}").show()

In [ ]:
for window in [5, 10, 15, None]:
    result = mir_eval.hierarchy.evaluate(ref.itvls, ref.labels, est_mono.itvls, est_mono.labels, window=window)
    px_metrics_bar(result, title=f"window = {window}").show()